In [1]:
!pip install nnunetv2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.3 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [3]:
import os

os.environ["nnUNet_raw"] = "/kaggle/working/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/kaggle/working/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/kaggle/working/nnUNet_results"


In [6]:
!nnUNetv2_print_available_pretrained_models


/bin/bash: line 1: nnUNetv2_print_available_pretrained_models: command not found


In [28]:
import nnunetv2

In [29]:
!pip show nnunetv2


Name: nnunetv2
Version: 2.6.2
Summary: nnU-Net is a framework for out-of-the box image segmentation.
Home-page: https://github.com/MIC-DKFZ/nnUNet
Author: Helmholtz Imaging Applied Computer Vision Lab
Author-email: Fabian Isensee <f.isensee@dkfz-heidelberg.de>
License: Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i)

In [5]:
!nnUNetv2_create_dataset_json -d 700 -i imagesTr -l labelsTr -n "KiTS23" --modalities CT --labels kidney tumor


/bin/bash: line 1: nnUNetv2_create_dataset_json: command not found


In [11]:
!python -m nnunetv2.experiment_planning.dataset_conversion.nnUNetv2_create_dataset_json \
    -d 700 \
    -i /kaggle/input/sample-nnunet-data/nnUNet_raw/Dataset700_KiTS23/imagesTr \
    -l /kaggle/input/sample-nnunet-data/nnUNet_raw/Dataset700_KiTS23/labelsTr \
    -n "KiTS23" \
    --modalities CT \
    --labels kidney tumor


/usr/bin/python3: Error while finding module specification for 'nnunetv2.experiment_planning.dataset_conversion.nnUNetv2_create_dataset_json' (ModuleNotFoundError: No module named 'nnunetv2.experiment_planning.dataset_conversion')


In [13]:
import os

os.environ["nnUNet_raw"] = "/kaggle/input/nnunet-sample-data/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/kaggle/working/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/kaggle/working/nnUNet_results"

# Confirm
for k, v in os.environ.items():
    if k.startswith("nnUNet"):
        print(k, "=", v)


nnUNet_raw = /kaggle/input/nnunet-sample-data/nnUNet_raw
nnUNet_preprocessed = /kaggle/working/nnUNet_preprocessed
nnUNet_results = /kaggle/working/nnUNet_results


In [14]:
!nnUNetv2_plan_and_preprocess -d 700 --verify_dataset_integrity


Fingerprint extraction...
Dataset700_KiTS23
Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_plan_and_preprocess", line 8, in <module>
    sys.exit(plan_and_preprocess_entry())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_entrypoints.py", line 180, in plan_and_preprocess_entry
    extract_fingerprints(args.d, args.fpe, args.npfp, args.verify_dataset_integrity, args.clean, args.verbose)
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 47, in extract_fingerprints
    extract_fingerprint_dataset(d, fingerprint_extractor_class, num_processes, check_dataset_integrity, clean,
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 30, in extract_fingerprint_dataset
    verify_dataset_integrity(join(nnUNet_raw, dataset_name), num_processes)
  File "/usr/local/lib/pytho

In [16]:
import os
import shutil
from pathlib import Path

imagesTr_dir = "/kaggle/input/nnunet-sample-data/nnUNet_raw/Dataset700_KiTS23/imagesTr"
labelsTr_dir = "/kaggle/input/nnunet-sample-data/nnUNet_raw/Dataset700_KiTS23/labelsTr"

# Flatten imagesTr
for i, folder in enumerate(os.listdir(imagesTr_dir)):
    folder_path = os.path.join(imagesTr_dir, folder)
    if os.path.isdir(folder_path):
        # Find the .nii or .nii.gz file inside the folder
        nii_files = list(Path(folder_path).glob("*.nii*"))
        if len(nii_files) == 0:
            continue
        nii_file = nii_files[0]
        # New name: case_00000_0000.nii.gz
        new_name = f"case_{i:05d}_0000.nii.gz"
        new_path = os.path.join(imagesTr_dir, new_name)
        shutil.move(str(nii_file), new_path)
        # Remove now-empty folder
        shutil.rmtree(folder_path)

# Flatten labelsTr similarly if needed
for i, file in enumerate(os.listdir(labelsTr_dir)):
    file_path = os.path.join(labelsTr_dir, file)
    if os.path.isdir(file_path):
        nii_files = list(Path(file_path).glob("*.nii*"))
        if len(nii_files) == 0:
            continue
        nii_file = nii_files[0]
        new_name = f"case_{i:05d}.nii.gz"
        new_path = os.path.join(labelsTr_dir, new_name)
        shutil.move(str(nii_file), new_path)
        shutil.rmtree(file_path)

print("✅ Flattened imagesTr and labelsTr, ready for nnU-Net preprocessing")


OSError: [Errno 30] Read-only file system: '/kaggle/input/nnunet-sample-data/nnUNet_raw/Dataset700_KiTS23/imagesTr/case_00000_0000.nii.gz'

In [41]:
import os
import shutil
from pathlib import Path

# Paths
src_dataset = "/kaggle/input/d/hansanikaumadi/nnunet-sample-data/nnUNet_raw/Dataset700_KiTS23"
dst_dataset = "/kaggle/working/"

# Copy entire dataset to working folder
shutil.copytree(src_dataset, dst_dataset)

imagesTr_dir = os.path.join(dst_dataset, "imagesTr")
labelsTr_dir = os.path.join(dst_dataset, "labelsTr")

# Flatten imagesTr
for i, folder in enumerate(os.listdir(imagesTr_dir)):
    folder_path = os.path.join(imagesTr_dir, folder)
    if os.path.isdir(folder_path):
        nii_files = list(Path(folder_path).glob("*.nii*"))
        if len(nii_files) == 0:
            continue
        nii_file = nii_files[0]
        new_name = f"case_{i:05d}_0000.nii.gz"
        new_path = os.path.join(imagesTr_dir, new_name)
        shutil.move(str(nii_file), new_path)
        shutil.rmtree(folder_path)

# Flatten labelsTr if labels are also in subfolders
for i, file in enumerate(os.listdir(labelsTr_dir)):
    file_path = os.path.join(labelsTr_dir, file)
    if os.path.isdir(file_path):
        nii_files = list(Path(file_path).glob("*.nii*"))
        if len(nii_files) == 0:
            continue
        nii_file = nii_files[0]
        new_name = f"case_{i:05d}.nii.gz"
        new_path = os.path.join(labelsTr_dir, new_name)
        shutil.move(str(nii_file), new_path)
        shutil.rmtree(file_path)

print("✅ Dataset copied to /kaggle/working, flattened, ready for nnU-Net preprocessing")


FileExistsError: [Errno 17] File exists: '/kaggle/working/'

In [39]:
!ls /kaggle/working/nnUNet_raw/Dataset700_KiTS23


ls: cannot access '/kaggle/working/nnUNet_raw/Dataset700_KiTS23': No such file or directory


In [22]:
import json

dataset_json_path = os.path.join(dst_dataset, "dataset.json")

dataset = {
    "labels": {"0": "background", "1": "kidney", "2": "tumor"},
    "channel_names": {"0": "CT"},
    "numTraining": 10,
    "file_ending": ".nii.gz"
}

with open(dataset_json_path, "w") as f:
    json.dump(dataset, f, indent=4)

print("✅ dataset.json created at:", dataset_json_path)


✅ dataset.json created at: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/dataset.json


In [23]:
!nnUNetv2_plan_and_preprocess -d 700 --verify_dataset_integrity


Fingerprint extraction...
Dataset700_KiTS23
Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_plan_and_preprocess", line 8, in <module>
    sys.exit(plan_and_preprocess_entry())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_entrypoints.py", line 180, in plan_and_preprocess_entry
    extract_fingerprints(args.d, args.fpe, args.npfp, args.verify_dataset_integrity, args.clean, args.verbose)
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 47, in extract_fingerprints
    extract_fingerprint_dataset(d, fingerprint_extractor_class, num_processes, check_dataset_integrity, clean,
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 30, in extract_fingerprint_dataset
    verify_dataset_integrity(join(nnUNet_raw, dataset_name), num_processes)
  File "/usr/local/lib/pytho

In [25]:
!ls /kaggle/working/nnUNet_raw/Dataset700_KiTS23


dataset.json  imagesTr	labelsTr


In [50]:
!cd /kaggle/working/
!rm -rf *

**New Approach**

In [52]:
import os
import shutil
import json
from pathlib import Path

# ------------------------------
# 1️⃣ Paths
# ------------------------------
src_dataset = "/kaggle/input/d/hansanikaumadi/nnunet-sample-data/nnUNet_raw/Dataset700_KiTS23"
dst_dataset = "/kaggle/working/nnUNet_raw/Dataset700_KiTS23"

imagesTr_dir = os.path.join(dst_dataset, "imagesTr")
labelsTr_dir = os.path.join(dst_dataset, "labelsTr")

# ------------------------------
# 2️⃣ Copy dataset to working directory (writable)
# ------------------------------
shutil.copytree(src_dataset, dst_dataset, dirs_exist_ok=True)
print("✅ Dataset copied to working folder:", dst_dataset)

# ------------------------------
# 3️⃣ Flatten imagesTr and labelsTr folders if needed
# ------------------------------
def flatten_folder(folder_path, prefix):
    folder_path = Path(folder_path)
    for i, subfolder in enumerate(folder_path.iterdir()):
        if subfolder.is_dir():
            # find .nii or .nii.gz files inside
            for f in subfolder.glob("*.nii*"):
                new_name = f"{prefix}_{i:03d}_0000.nii.gz" if "images" in folder_path.name else f"{prefix}_{i:03d}.nii.gz"
                shutil.move(str(f), folder_path / new_name)
            # remove now-empty folder
            shutil.rmtree(subfolder)

flatten_folder(imagesTr_dir, "imaging")
flatten_folder(labelsTr_dir, "segmentation")
print("✅ Flattened imagesTr and labelsTr folders")

# ------------------------------
# 4️⃣ Create dataset.json
# ------------------------------
dataset_json_path = os.path.join(dst_dataset, "dataset.json")

training_list = []
for i in range(10):  # adjust if more samples
    training_list.append({
        "image": f"./imagesTr/imaging_{i:03d}_0000.nii.gz",
        "label": f"./labelsTr/segmentation_{i:03d}.nii.gz"
    })

dataset = {
    "labels": {"0": "background", "1": "kidney", "2": "tumor"},
    "channel_names": {"0": "CT"},
    "numTraining": len(training_list),
    "file_ending": ".nii.gz",
    "training": training_list
}

with open(dataset_json_path, "w") as f:
    json.dump(dataset, f, indent=4)

print("✅ dataset.json created at:", dataset_json_path)

# # ------------------------------
# # 5️⃣ Run nnU-Net preprocessing
# # ------------------------------
# !nnUNetv2_plan_and_preprocess -d 700 --verify_dataset_integrity --nnunet_raw /kaggle/working/nnUNet_raw


✅ Dataset copied to working folder: /kaggle/working/nnUNet_raw/Dataset700_KiTS23
✅ Flattened imagesTr and labelsTr folders
✅ dataset.json created at: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/dataset.json


In [53]:
!ls /kaggle/working/nnUNet_raw/Dataset700_KiTS23


dataset.json  imagesTr	labelsTr


In [47]:
!nnUNetv2_plan_and_preprocess -d 700 --verify_dataset_integrity


Fingerprint extraction...
Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_plan_and_preprocess", line 8, in <module>
    sys.exit(plan_and_preprocess_entry())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_entrypoints.py", line 180, in plan_and_preprocess_entry
    extract_fingerprints(args.d, args.fpe, args.npfp, args.verify_dataset_integrity, args.clean, args.verbose)
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 47, in extract_fingerprints
    extract_fingerprint_dataset(d, fingerprint_extractor_class, num_processes, check_dataset_integrity, clean,
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 26, in extract_fingerprint_dataset
    dataset_name = convert_id_to_dataset_name(dataset_id)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File

In [54]:
import os

os.environ["nnUNet_raw"] = "/kaggle/working/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/kaggle/working/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/kaggle/working/nnUNet_results"


In [ ]:
!unzip wefwfgth -d dbeghhyj

In [65]:
!mv /kaggle/working/nnUNet_raw/Dataset700_KiTS23/labelsTr/segmentation_009.nii.gz /kaggle/working/nnUNet_raw/Dataset700_KiTS23/labelsTr/imaging_009.nii.gz

In [68]:
!nnUNetv2_plan_and_preprocess -d 700 --verify_dataset_integrity


Fingerprint extraction...
Dataset700_KiTS23
Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_plan_and_preprocess", line 8, in <module>
    sys.exit(plan_and_preprocess_entry())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_entrypoints.py", line 180, in plan_and_preprocess_entry
    extract_fingerprints(args.d, args.fpe, args.npfp, args.verify_dataset_integrity, args.clean, args.verbose)
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 47, in extract_fingerprints
    extract_fingerprint_dataset(d, fingerprint_extractor_class, num_processes, check_dataset_integrity, clean,
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 30, in extract_fingerprint_dataset
    verify_dataset_integrity(join(nnUNet_raw, dataset_name), num_processes)
  File "/usr/local/lib/pytho

In [67]:
import json
import os

# Path to your dataset.json
dataset_json_path = "/kaggle/working/nnUNet_raw/Dataset700_KiTS23/dataset.json"

# 1️⃣ Load the JSON
with open(dataset_json_path, "r") as f:
    dataset = json.load(f)

# 2️⃣ Replace "segmentation_" with "imaging_" in each training label
if "training" in dataset:
    for entry in dataset["training"]:
        if "label" in entry and "segmentation_" in entry["label"]:
            entry["label"] = entry["label"].replace("segmentation_", "imaging_")

# 3️⃣ Save updated JSON
with open(dataset_json_path, "w") as f:
    json.dump(dataset, f, indent=4)

print("✅ Updated dataset.json saved at:", dataset_json_path)


✅ Updated dataset.json saved at: /kaggle/working/nnUNet_raw/Dataset700_KiTS23/dataset.json
